In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
%matplotlib inline
import seaborn as sns
sns.set_context('poster')

## Moving Average Models With MSA, Years and Features

### Import, Split, and Standardize Data

In [2]:
start = datetime.datetime.time(datetime.datetime.now())

In [3]:
df = pd.read_csv('../data/merged/all_data_2006_to_2016.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 928 entries, 0 to 927
Data columns (total 13 columns):
MSA_orig                                                                       928 non-null object
MSA_corr                                                                       928 non-null object
MSA_abbr                                                                       928 non-null object
year                                                                           928 non-null int64
now_married_except_separated                                                   928 non-null float64
less_than_high_school_diploma                                                  928 non-null float64
unmarried_portion_of_women_15_to_50_years_who_had_a_birth_in_past_12_months    928 non-null float64
households_with_food_stamp_snap_benefits                                       928 non-null float64
percentage_married-couple_family                                               928 non-null float64


In [5]:
# drop extra MSA names
df = df.drop(['MSA_orig', 'MSA_corr'], axis=1)

In [6]:
# remove outliers with atypically high murder rates
df = df[df.MSA_abbr != 'NEW_ORLEANS_LA']
df = df[(df.MSA_abbr != 'MEMPHIS_TN') | (df.year != 2016)]
df = df[(df.MSA_abbr != 'BATON_ROUGE_LA') | (df.year != 2007)]

In [7]:
# preserve 'MSA_abbr'
df_msa = df['MSA_abbr']

In [8]:
df = pd.get_dummies(df, columns=['MSA_abbr'], drop_first=False)

In [9]:
# put back 'MSA_abbr'
df['MSA_abbr'] = df_msa.values

In [10]:
# move 'MSA_abbr' from last column to first column
cols = [df.columns[-1]] + [col for col in df if col != df.columns[-1]]
df = df[cols]

In [11]:
df.head()

,MSA_abbr,year,now_married_except_separated,less_than_high_school_diploma,unmarried_portion_of_women_15_to_50_years_who_had_a_birth_in_past_12_months,households_with_food_stamp_snap_benefits,percentage_married-couple_family,percentage_female_householder_no_husband_present_family,poverty_all_people,house_median_value_(dollars),...,MSA_abbr_TAMPA_FL,MSA_abbr_TOLEDO_OH,MSA_abbr_TUCSON_AZ,MSA_abbr_TULSA_OK,MSA_abbr_VIRGINIA_BEACH_NC,MSA_abbr_WASHINGTON_DC,MSA_abbr_WICHITA_KS,MSA_abbr_WINSTON_NC,MSA_abbr_WORCESTER_MA,MSA_abbr_YOUNGSTOWN_OH
0,ATLANTA_GA,2006,49.2,14.2,34.3,5.9,72.6,20.0,11.9,186800,...,0,0,0,0,0,0,0,0,0,0
1,AUSTIN_TX,2006,48.7,13.7,30.9,5.9,75.0,17.0,13.0,164100,...,0,0,0,0,0,0,0,0,0,0
2,BALTIMORE_MD,2006,47.2,14.0,31.4,5.5,71.8,21.4,9.0,300600,...,0,0,0,0,0,0,0,0,0,0
3,BIRMINGHAM_AL,2006,50.9,15.8,34.7,7.1,72.5,21.6,13.1,131400,...,0,0,0,0,0,0,0,0,0,0
4,BUFFALO_NY,2006,47.1,12.9,38.0,9.6,72.8,20.9,14.2,105000,...,0,0,0,0,0,0,0,0,0,0


In [12]:
list(df)[:15]

['MSA_abbr',
 'year',
 'now_married_except_separated',
 'less_than_high_school_diploma',
 'unmarried_portion_of_women_15_to_50_years_who_had_a_birth_in_past_12_months',
 'households_with_food_stamp_snap_benefits',
 'percentage_married-couple_family',
 'percentage_female_householder_no_husband_present_family',
 'poverty_all_people',
 'house_median_value_(dollars)',
 'murder_per_100_k',
 'MSA_abbr_AKRON_OH',
 'MSA_abbr_ALBANY_NY',
 'MSA_abbr_ALBUQUERQUE_NM',
 'MSA_abbr_ALLENTOWN_PA']

In [13]:
# train test split / separate labels and features

label_col = 'murder_per_100_k'

split_yr = 2011

df_train_s = df[df['year'] <= split_yr]
df_test = df[df['year'] > split_yr]

print('Len train: {}'.format(len(df_train_s)))
print('Len test: {}'.format(len(df_test)))

Len train: 490
Len test: 426


In [14]:
# smooth the training data (features and response variable)
# by averaging, for each observation, the values within each MSA
# across a moving window of [ma_yrs] years.
def moving_average(df, ma_yrs):
    
    # the min and max years in the dataset
    # set the starting year of the earliest window
    # and the ending year of the latest window
    min_yr = df.year.min()
    max_yr = df.year.max()
    
    # preserve columns
    cols = list(df)
    
    # get the list of unique MSAs
    # to cycle through
    MSAs = df.MSA_abbr.unique()
    
    # here is where we will store the smoothed dataset
    main_df = []
    
    # look for each possible year within the MSA
    # to see whether it exists; if yes, create the
    # smoothed data for it
    for yr in range(min_yr, max_yr+1):
        
        # set the boundaries of the current window
        window_min_yr = yr - int(ma_yrs/2)
        window_max_yr = yr + int(ma_yrs/2)
        
        # create, for each MSA, the smoothed data
        # for each year present in the MSA
        for MSA in MSAs:
            
            # does the MSA have the row for this year?...
            MSA_row = np.array(df[(df.MSA_abbr == MSA) & (df.year == yr)])
            
            # ... if not, skip to the next MSA
            # otherwise, create the smoothed data for the MSA-year
            if MSA_row.size == 0:
                continue
                
            # preserve the one-hot-encoded data
            dummies = MSA_row[:, 11:]
            
            # retrieve, for this MSA, the set of observations within the window
            # for the current year
            MSA_row = np.array(df[(df.MSA_abbr == MSA) & (df.year >= window_min_yr) & (df.year <= window_max_yr)])
            
            # leave off the one-hot-encoded data
            # and also leave off the MSA and the year
            MSA_row = MSA_row[:, 2:11]
            
            # smooth (i.e., average) the data for the current MSA-year
            MSA_row = np.mean(MSA_row, axis=0)
            
            # append the original year to the smoothed row
            # (we don't want the year value itself smoothed!)
            MSA_row = np.append(yr, MSA_row)
            
            # append the MSA name to the smoothed row
            MSA_row = np.append(MSA, MSA_row)
            
            # append the one-hot-encoded data to the smoothed row
            MSA_row = np.append(MSA_row, dummies)
            
            # append the completed, smoothed row to the smoothed dataset
            main_df.append(MSA_row)
    
    # make the dataset a pandas dataframe
    main_df = pd.DataFrame(main_df)
    
    # assign the column names
    main_df.columns = cols
    
    # return the smoothed dataframe, with column names
    return main_df

In [15]:
df_train_s_ma = moving_average(df_train_s, 5)

In [16]:
df_train_s_ma.head()

,MSA_abbr,year,now_married_except_separated,less_than_high_school_diploma,unmarried_portion_of_women_15_to_50_years_who_had_a_birth_in_past_12_months,households_with_food_stamp_snap_benefits,percentage_married-couple_family,percentage_female_householder_no_husband_present_family,poverty_all_people,house_median_value_(dollars),...,MSA_abbr_TAMPA_FL,MSA_abbr_TOLEDO_OH,MSA_abbr_TUCSON_AZ,MSA_abbr_TULSA_OK,MSA_abbr_VIRGINIA_BEACH_NC,MSA_abbr_WASHINGTON_DC,MSA_abbr_WICHITA_KS,MSA_abbr_WINSTON_NC,MSA_abbr_WORCESTER_MA,MSA_abbr_YOUNGSTOWN_OH
0,ATLANTA_GA,2006,49.200000,13.466667,34.500000,6.233333,72.600000,20.533333,11.600000,193600.000000,...,0,0,0,0,0,0,0,0,0,0
1,AUSTIN_TX,2006,48.633333,13.833333,30.000000,5.866667,75.266667,17.166667,12.833333,176700.000000,...,0,0,0,0,0,0,0,0,0,0
2,BALTIMORE_MD,2006,46.866667,13.233333,34.066667,5.600000,71.566667,21.600000,9.166667,307900.000000,...,0,0,0,0,0,0,0,0,0,0
3,BIRMINGHAM_AL,2006,50.833333,15.633333,36.766667,7.700000,72.533333,21.700000,12.933333,139466.666667,...,0,0,0,0,0,0,0,0,0,0
4,BUFFALO_NY,2006,46.800000,12.100000,40.900000,10.366667,72.233333,21.200000,13.666667,109733.333333,...,0,0,0,0,0,0,0,0,0,0


In [17]:
# train test split / separate labels and features

label_col = 'murder_per_100_k'

split_yr = 2014

x_train_s = df_train_s_ma.drop([label_col], axis=1).drop('MSA_abbr', axis=1)
x_test_s = df_test.drop([label_col], axis=1).drop('MSA_abbr', axis=1)
y_train = df_train_s_ma[label_col]
y_test = df_test[label_col]

print('Sizes match: {}'.format(len(x_train_s)==len(y_train)))
print()
print('Len x_train: {}'.format(len(x_train_s)))
print('Len x_test: {}'.format(len(x_test_s)))
print('Len y_train: {}'.format(len(y_train)))
print('Len x_test: {}'.format(len(y_test)))

Sizes match: True

Len x_train: 490
Len x_test: 426
Len y_train: 490
Len x_test: 426


In [18]:
# standardize data

from sklearn.preprocessing import StandardScaler
standardizer = StandardScaler().fit(x_train_s)

x_train = standardizer.transform(x_train_s)
x_test = standardizer.transform(x_test_s)

### Simple Linear Regression

In [19]:
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, BayesianRidge, HuberRegressor
from sklearn.model_selection import GridSearchCV

In [20]:
# instantiate and fit models

def make_models(x_train, y_train):
    md = dict()

    md['linear'] = LinearRegression().fit(x_train, y_train)
    md['ridge'] = RidgeCV(cv=15).fit(x_train, y_train)
    md['lasso'] = LassoCV(cv=15).fit(x_train, y_train)
    md['bayes'] = BayesianRidge(tol=0.0001).fit(x_train, y_train)
    md['huber'] = GridSearchCV(HuberRegressor(),{'epsilon': [1.0,1.1,1.2,1.3,1.4,1.5,1.6,1.7]}).fit(x_train, y_train).best_estimator_
    
    return md

In [21]:
# score models

def score_model(model):

    train_score = model.score(x_train, y_train)
    test_score = model.score(x_test, y_test)

    return np.array([train_score, test_score])

In [22]:
def results_df(model):

    # get train and test scores
    scores_df = pd.DataFrame(score_model(model)).transpose()
    scores_df.columns = ['Train R2','Test R2']

    # get coefficient matrix
    coeffs_df = pd.DataFrame(model.coef_).transpose()
    coeffs_df.columns = x_train_s.columns

    # join dataframes
    return pd.concat([scores_df, coeffs_df], axis=1)

In [23]:
from sklearn.utils import resample

In [24]:
def run_experiment(n_iters):
    
    sample_results = dict()
    
    for n in range(n_iters):
        # get new sample
        xb, yb = resample(x_train, y_train)

        # make and fit models
        model_dict = make_models(xb, yb)

        # get sample of results for each model
        for key in model_dict:
            
            # initialize empty dictionary
            if key not in sample_results:
                sample_results[key] = []
                        
            # get model results
            sample_results[key].append(results_df(model_dict[key]))
            
    # concatenate results dfs into single df
    for key in sample_results:
        sample_results[key] = pd.concat(sample_results[key])
        
    return sample_results

In [25]:
exp = run_experiment(10)

/Users/ilanjdor/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/ilanjdor/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/ilanjdor/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
/Users/ilanjdor/anaconda/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:484: ConvergenceWarning: Objective di

In [26]:
coef_dict = dict()

# iterate over all models
for key in exp:
    
    # iterate over results of this model
    for c in exp[key].columns:
        
        # initialize dict for result names
        if c not in coef_dict:
            coef_dict[c] = dict()
    
        # add this coeff to the dict
        coef_dict[c][key] = exp[key][c]

# convert dict of dicts into dict of dataframes
coef_dfs = {key: pd.DataFrame(coef_dict[key]) for key in coef_dict}

In [27]:
for key in exp:
    print(key)
    print()
    print(exp[key].mean()[:2])
    print()

linear

Train R2   -8.181077e+24
Test R2    -1.180159e+25
dtype: float64

ridge

Train R2    0.969928
Test R2     0.610709
dtype: float64

lasso

Train R2    0.969375
Test R2     0.584382
dtype: float64

bayes

Train R2    0.970579
Test R2     0.617515
dtype: float64

huber

Train R2    0.910522
Test R2     0.541203
dtype: float64



In [28]:
def print_runtime():
    hours = int(str(end)[0:2])-int(str(start)[0:2])
    minutes = int(str(end)[3:5])-int(str(start)[3:5])
    seconds = int(str(end)[6:8])-int(str(start)[6:8])
    if hours < 0:
        hours = hours + 24
    if minutes < 0:
        minutes = minutes + 60
        hours = hours - 1
    if seconds < 0:
        seconds = seconds + 60
        minutes = minutes - 1
    print(hours, "hrs", minutes, "mins", seconds, "secs")

In [29]:
end = datetime.datetime.time(datetime.datetime.now())

In [30]:
print_runtime()

0 hrs 0 mins 47 secs
